In [1]:
import fibre
import numpy as np

from __future__ import print_function
logger = fibre.utils.Logger(verbose=True)

## 机械臂的SN码可以通过`lsusb`命令查看

In [2]:
teach_arm = fibre.find_any(serial_number="208C31875253", logger=logger)

USB discover loop
ConfigurationValue 1
	InterfaceNumber 0,0
		EndpointAddress 130
	InterfaceNumber 1,0
		EndpointAddress 1
		EndpointAddress 129
	InterfaceNumber 2,0
		EndpointAddress 3
		EndpointAddress 131

Kernel Driver was not attached
EndpointAddress for writing 3
EndpointAddress for reading 131
Connecting to device on USB device bus 0 device 1
JSON: [
{"name":"","id":0,"type":"json","access":"r"},
{"name":"serial_number","id":1,"type":"uint64","access":"r"},
{"name":"get_temperature","id":2,"type":"function","inputs":[],"outputs":[
{"name":"result","id":3,"type":"float","access":"rw"}]},
{"name":"get_voltage","id":4,"type":"function","inputs":[],"outputs":[
{"name":"result","id":5,"type":"float","access":"rw"}]},
{"name":"robot","type":"object","members":[
{"name":"calibrate_home_offset","id":6,"type":"function","inputs":[],"outputs":[]},
{"name":"homing","id":7,"type":"function","inputs":[],"outputs":[]},
{"name":"resting","id":8,"type":"function","inputs":[],"outputs":[]},
{"na

## RPC连上后,每一个层级都可以直接打印查看

In [3]:
teach_arm

serial_number = 208C31875253 (int)
get_temperature()
get_voltage()
robot:
  calibrate_home_offset()
  homing()
  resting()
  joint_1: ...
  joint_2: ...
  joint_3: ...
  joint_4: ...
  joint_5: ...
  joint_6: ...
  joint_all: ...
  hand: ...
  reboot()
  set_enable(enable: bool)
  set_rgb_enable(enable: bool)
  set_rgb_mode(mode: int)
  move_j(j1: float, j2: float, j3: float, j4: float, j5: float, j6: float)
  move_l(x: float, y: float, z: float, a: float, b: float, c: float)
  set_joint_speed(speed: float)
  set_joint_acc(acc: float)
  set_command_mode(mode: int)
  tuning: ...
  eef_pose: ...

In [4]:
teach_arm.robot

calibrate_home_offset()
homing()
resting()
joint_1:
  angle = 0.0044531249441206455 (float)
  velocity = 0.0 (float)
  reboot()
  get_temperature()
  set_enable_temperature(enable: bool)
  erase_configs()
  set_enable(enable: bool)
  set_position_with_time(pos: float, time: float)
  set_position(pos: float)
  set_velocity(vel: float)
  set_velocity_limit(vel: float)
  set_current(current: float)
  set_current_limit(current: float)
  set_node_id(id: int)
  set_acceleration(acc: float)
  apply_home_offset()
  do_calibration()
  set_enable_on_boot(enable: bool)
  set_dce_kp(vel: int)
  set_dce_kv(vel: int)
  set_dce_ki(vel: int)
  set_dce_kd(vel: int)
  set_enable_stall_protect(enable: bool)
  update_angle()
joint_2:
  angle = 0.2491406351327896 (float)
  velocity = 0.0 (float)
  reboot()
  get_temperature()
  set_enable_temperature(enable: bool)
  erase_configs()
  set_enable(enable: bool)
  set_position_with_time(pos: float, time: float)
  set_position(pos: float)
  set_velocity(vel: fl

In [4]:
teach_arm.robot.joint_1

angle = 1.896328091621399 (float)
velocity = 0.0 (float)
reboot()
get_temperature()
set_enable_temperature(enable: bool)
erase_configs()
set_enable(enable: bool)
set_position_with_time(pos: float, time: float)
set_position(pos: float)
set_velocity(vel: float)
set_velocity_limit(vel: float)
set_current(current: float)
set_current_limit(current: float)
set_node_id(id: int)
set_acceleration(acc: float)
apply_home_offset()
do_calibration()
set_enable_on_boot(enable: bool)
set_dce_kp(vel: int)
set_dce_kv(vel: int)
set_dce_ki(vel: int)
set_dce_kd(vel: int)
set_enable_stall_protect(enable: bool)
update_angle()

In [5]:
teach_arm.robot.hand

angle = 36.86000061035156 (float)
get_mode()
get_current_limit()
set_angle(angle: float)
set_mode(mode: int)
set_current_limit(current: float)
set_torque(current: float)
enable_close_loop()
save()

## 接着让我们来测试一些功能

### 机械臂MoveJ

In [3]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_j(0, 0, 90, 0, 0, 0)

True

### 获取所有关节角度(获取到的这个角度是电机的关节机械角, 和运动学模型角度之间存在一个固定的offset)

In [8]:
joint_offset = np.array([0,-73,180,0,0,0])

In [9]:
all_joint_angles = teach_arm.robot.joint_1.angle, teach_arm.robot.joint_2.angle, teach_arm.robot.joint_3.angle, teach_arm.robot.joint_4.angle, teach_arm.robot.joint_5.angle, teach_arm.robot.joint_6.angle
all_joint_angles = np.round(np.array(all_joint_angles))
all_joint_angles = all_joint_angles + joint_offset
all_joint_angles

array([ 0.,  0., 90.,  0.,  0.,  0.])

### 获取当前末端位姿(旋转使用ZYX欧拉角序列(也称为RPY - Roll, Pitch, Yaw))

In [4]:
def get_current_pose():
    teach_arm.robot.eef_pose.update_pose_6D()
    x,y,z,rz,ry,rx = teach_arm.robot.eef_pose.x, teach_arm.robot.eef_pose.y, teach_arm.robot.eef_pose.z, teach_arm.robot.eef_pose.a, teach_arm.robot.eef_pose.b, teach_arm.robot.eef_pose.c
    current_pose = np.array([x,y,z,rz,ry,rx])
    return current_pose

current_pose = get_current_pose()
current_pose


array([ 2.27499405e+02,  0.00000000e+00,  3.24500549e+02, -2.10649698e-09,
        9.00000000e+01,  0.00000000e+00])

### 机械臂MoveL(直接控制末端在笛卡尔空间运动, 输入单位是mm)

In [11]:
# 沿着x轴方向移动10mm
teach_arm.robot.move_l(current_pose[0]+10, current_pose[1], current_pose[2], current_pose[3], current_pose[4], current_pose[5])
current_pose = get_current_pose()
current_pose

array([227.49700928,   0.        , 324.50286865,   0.        ,
        90.        ,   0.        ])

In [12]:
# 沿着z轴方向移动10mm
teach_arm.robot.move_l(current_pose[0], current_pose[1], current_pose[2]+10, current_pose[3], current_pose[4], current_pose[5])
current_pose = get_current_pose()
current_pose


array([237.49632263,   0.        , 324.16833496,   0.        ,
        90.        ,   0.        ])

### 归巢

In [5]:
teach_arm.robot.resting()

### 复位回到home pose

In [13]:
teach_arm.robot.move_j(0, 0, 90, 0, 0, 0)

True

### 示教臂的夹爪实时位置读取

In [6]:
## 让夹爪电机进入零力矩状态
teach_arm.robot.hand.set_mode(0)
teach_arm.robot.hand.set_torque(0)

In [ ]:
teach_arm.robot.set_enable(False)
## 拖动夹爪然后查看角度变化
rate = 2  # 50Hz
def get_current_pose():
    teach_arm.robot.eef_pose.update_pose_6D()
    x,y,z,rz,ry,rx = teach_arm.robot.eef_pose.x, teach_arm.robot.eef_pose.y, teach_arm.robot.eef_pose.z, teach_arm.robot.eef_pose.a, teach_arm.robot.eef_pose.b, teach_arm.robot.eef_pose.c
    current_pose = np.array([x,y,z,rz,ry,rx])
    return current_pose


import time
while True:
    start_time = time.time()
    current_pose = get_current_pose()
    print(current_pose)
    time.sleep(1/rate)
    print(teach_arm.robot.hand.angle)

[ 98.47728729   9.13166904 167.75912476 115.73566437  77.91817474
 120.53395081]
36.72999954223633
[ 98.47797394   9.13173294 167.7580719  115.73976135  77.91777039
 120.53794098]
36.720001220703125
[ 98.47809601   9.13052273 167.7580719  115.73976135  77.91777039
 120.5372467 ]
36.689998626708984
[ 98.4773407    9.13045597 167.75668335 115.73863983  77.9178772
 120.53616333]
36.779998779296875
[ 98.47802734   9.13051891 167.75564575 115.74271393  77.91747284
 120.54013824]
37.68000030517578
[ 98.47711182   9.13165379 167.76147461 115.72241211  77.91947937
 120.52100372]
37.58000183105469
[ 68.57160187   6.35855389 277.07998657  19.2056179   59.62438583
  27.28635025]
37.11000061035156
[ 68.56789398   6.35820961 277.08081055  19.20456886  59.62308121
  27.28545189]
36.52000045776367
[ 56.64007187   5.25215816 294.4364624   15.41787434  54.04880905
  24.11119843]
36.54999923706055
[ 57.28276443   5.31624603 293.60244751  15.58285904  54.33105087
  24.24962997]
36.16999816894531
[ 88.811

KeyboardInterrupt: 

error in usbbulk_transport.py, process_packet
Traceback (most recent call last):
  File "/Users/jack/Desktop/dummy_ctrl/fibre/usbbulk_transport.py", line 127, in get_packet
    ret = self.epr.read(bufferLen, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 431, in read
    return self.device.read(self, size_or_buffer, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 1043, in read
    ret = fn(
          ^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 850, in bulk_read
    return self.__read(self.lib.libusb_bulk_transfer,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 958, in __read
    _check(retval)
  File "/Use

37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.56999969482422
37.5699996

KeyboardInterrupt: 

In [ ]:
teach_arm.robot.set_enable(False)
import time
rate = 0.1  # 10Hz
## 让夹爪电机进入零力矩状态
teach_arm.robot.hand.set_mode(0)
teach_arm.robot.hand.set_torque(0)
while True:
    start_time = time.time()
    joint1_angle = teach_arm.robot.joint_1.angle
    joint2_angle = teach_arm.robot.joint_2.angle
    joint3_angle = teach_arm.robot.joint_3.angle
    joint4_angle = teach_arm.robot.joint_4.angle
    joint5_angle = teach_arm.robot.joint_5.angle
    joint6_angle = teach_arm.robot.joint_6.angle
    # print(f"Current leader Joints: {joint1_angle}, {joint2_angle}, {joint3_angle}, {joint4_angle}, {joint5_angle}, {joint6_angle}")
    all_joint_angles = np.array([joint1_angle, joint2_angle, joint3_angle, joint4_angle, joint5_angle, joint6_angle]) + joint_offset
    all_joint_angles = np.round(all_joint_angles, 0)
    # print("Current gripper: ",teach_arm.robot.hand.angle)
    # joint1_qpos = teach_arm.robot.joint_1.position
    joint1_qvel = teach_arm.robot.joint_1.velocity
    print( "qvel: ", joint1_qvel)
    joint2_qvel = teach_arm.robot.joint_2.velocity
    joint3_qvel = teach_arm.robot.joint_3.velocity
    joint4_qvel = teach_arm.robot.joint_4.velocity
    joint5_qvel = teach_arm.robot.joint_5.velocity
    joint6_qvel = teach_arm.robot.joint_6.velocity
    print( "qvel: ", joint2_qvel, joint3_qvel, joint4_qvel, joint5_qvel, joint6_qvel)
    # follow_arm.robot.move_j(all_joint_angles[0], all_joint_angles[1], all_joint_angles[2], all_joint_angles[3], all_joint_angles[4], all_joint_angles[5])
    # follow_arm.robot.hand.set_angle(teach_arm.robot.hand.angle - teach_hand_init_angle + follow_hand_init_angle)
    # print(f"Current follower Joints (state): {all_joint_angles}")
    # collector.collect_step(
    # teach_joints=np.array([joint1_angle, joint2_angle, joint3_angle, 
    #                         joint4_angle, joint5_angle, joint6_angle]),
    # follow_joints=all_joint_angles,
    # teach_gripper=teach_arm.robot.hand.angle,
    # follow_gripper=follow_arm.robot.hand.angle
    # )
    # Sleep precisely to maintain 10Hz
    elapsed = time.time() - start_time
    if elapsed < rate:
        time.sleep(rate - elapsed)
    
    # Calculate and print actual loop frequency
    loop_time = time.time() - start_time
    actual_freq = 1.0 / loop_time if loop_time > 0 else 0
    print(f"Current loop frequency: {actual_freq:.2f} Hz")

qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.81 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.88 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.36 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.56 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.52 Hz
qvel:  0.0
qvel:  0.0 0.0 0.0 0.0 0.0
Current loop frequency: 9.92 Hz
qvel:  0.0
qvel:  0.

KeyboardInterrupt: 

### 示教臂的夹爪实时力矩读取(即将支持,硬件施工中)

### Follow Arm相机测试

In [ ]:
import cv2
mjpg_streamer_url = "http://192.168.65.110:8080/?action=stream"
cap = cv2.VideoCapture(mjpg_streamer_url)
while True:
    ret, frame = cap.read()
    cv2.imshow("frame", frame)
    cv2.waitKey(1)
cv2.destroyAllWindows()

2025-03-21 15:16:01.361 python[63765:10187927] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-21 15:16:01.361 python[63765:10187927] +[IMKInputSession subclass]: chose IMKInputSession_Modern


KeyboardInterrupt: 

error in usbbulk_transport.py, process_packet
Traceback (most recent call last):
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/fibre/usbbulk_transport.py", line 127, in get_packet
    ret = self.epr.read(bufferLen, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 431, in read
    return self.device.read(self, size_or_buffer, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 1043, in read
    ret = fn(
          ^^^
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 850, in bulk_read
    return self.__read(self.lib.libusb_bulk_transfer,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yinzi/Downl

# 校准机械臂初始状态

In [9]:
from __future__ import print_function
import fibre
logger = fibre.utils.Logger(verbose=True)
SN = "396636713233"
teach_arm = fibre.find_any(serial_number=SN, logger=logger)

ConfigurationValue 1
	InterfaceNumber 0,0
		EndpointAddress 130
	InterfaceNumber 1,0
		EndpointAddress 1
		EndpointAddress 129
	InterfaceNumber 2,0
		EndpointAddress 3
		EndpointAddress 131

Kernel Driver was not attached
EndpointAddress for writing 3
EndpointAddress for reading 131
Connecting to device on USB device bus 1 device 1
Device reported JSON version ID: 579930592
JSON: [
{'name': '', 'id': 0, 'type': 'json', 'access': 'r'}, 
{'name': 'serial_number', 'id': 1, 'type': 'uint64', 'access': 'r'}, 
{'name': 'get_temperature', 'id': 2, 'type': 'function', 'inputs': [], 'outputs': [
{'name': 'result', 'id': 3, 'type': 'float', 'access': 'rw'}]}, 
{'name': 'get_voltage', 'id': 4, 'type': 'function', 'inputs': [], 'outputs': [
{'name': 'result', 'id': 5, 'type': 'float', 'access': 'rw'}]}, 
{'name': 'robot', 'type': 'object', 'members': [
{'name': 'calibrate_home_offset', 'id': 6, 'type': 'function', 'inputs': [], 'outputs': []}, 
{'name': 'homing', 'id': 7, 'type': 'function', 'inpu

received unexpected ACK: 132


In [11]:
teach_arm.robot.resting()

In [4]:
# Enable motor but disable circle udpate
teach_arm.robot.joint_all.set_enable(True)
teach_arm.robot.set_enable(False)

## 先手动把机械臂挪成L姿态, 然后运行以下代码

In [5]:
teach_arm.robot.joint_1.apply_home_offset()
teach_arm.robot.joint_2.apply_home_offset()
teach_arm.robot.joint_3.apply_home_offset()
teach_arm.robot.joint_4.apply_home_offset()
teach_arm.robot.joint_5.apply_home_offset()
teach_arm.robot.joint_6.apply_home_offset()

## 再手动把机械臂挪回休息状态, 然后运行以下代码

In [ ]:
teach_arm.robot.joint_1.apply_home_offset()
teach_arm.robot.joint_2.apply_home_offset()
teach_arm.robot.joint_3.apply_home_offset()
teach_arm.robot.joint_4.apply_home_offset()
teach_arm.robot.joint_5.apply_home_offset()
teach_arm.robot.joint_6.apply_home_offset()

error in usbbulk_transport.py, process_packet
Traceback (most recent call last):
  File "/Users/jack/Desktop/dummy_ctrl/fibre/usbbulk_transport.py", line 127, in get_packet
    ret = self.epr.read(bufferLen, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 431, in read
    return self.device.read(self, size_or_buffer, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 1043, in read
    ret = fn(
          ^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 850, in bulk_read
    return self.__read(self.lib.libusb_bulk_transfer,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 958, in __read
    _check(retval)
  File "/Use

In [57]:
import fibre
import numpy as np

from __future__ import print_function
logger = fibre.utils.Logger(verbose=True)
teach_arm = fibre.find_any(serial_number="208C31875253", logger=logger)
follow_arm = fibre.find_any(serial_number="396636713233", logger=logger)

ConfigurationValue 1
	InterfaceNumber 0,0
		EndpointAddress 130
	InterfaceNumber 1,0
		EndpointAddress 1
		EndpointAddress 129
	InterfaceNumber 2,0
		EndpointAddress 3
		EndpointAddress 131



Kernel Driver was not attached
EndpointAddress for writing 3
EndpointAddress for reading 131
Connecting to device on USB device bus 0 device 1
Device reported JSON version ID: 579930592
JSON: [
{'name': '', 'id': 0, 'type': 'json', 'access': 'r'}, 
{'name': 'serial_number', 'id': 1, 'type': 'uint64', 'access': 'r'}, 
{'name': 'get_temperature', 'id': 2, 'type': 'function', 'inputs': [], 'outputs': [
{'name': 'result', 'id': 3, 'type': 'float', 'access': 'rw'}]}, 
{'name': 'get_voltage', 'id': 4, 'type': 'function', 'inputs': [], 'outputs': [
{'name': 'result', 'id': 5, 'type': 'float', 'access': 'rw'}]}, 
{'name': 'robot', 'type': 'object', 'members': [
{'name': 'calibrate_home_offset', 'id': 6, 'type': 'function', 'inputs': [], 'outputs': []}, 
{'name': 'homing', 'id': 7, 'type': 'function', 'inputs': [], 'outputs': []}, 
{'name': 'resting', 'id': 8, 'type': 'function', 'inputs': [], 'outputs': []}, 
{'name': 'joint_1', 'type': 'object', 'members': [
{'name': 'angle', 'id': 9, 'type':

error in usbbulk_transport.py, process_packet
Traceback (most recent call last):
  File "/Users/jack/Desktop/dummy_ctrl/fibre/usbbulk_transport.py", line 127, in get_packet
    ret = self.epr.read(bufferLen, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 431, in read
    return self.device.read(self, size_or_buffer, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 1043, in read
    ret = fn(
          ^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 850, in bulk_read
    return self.__read(self.lib.libusb_bulk_transfer,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 958, in __read
    _check(retval)
  File "/Use

In [68]:
teach_arm.robot.set_enable(True)
teach_arm.robot.hand.set_mode(0)

In [67]:
teach_arm.robot.hand.set_angle(27)

In [41]:
print(teach_arm.robot.hand.angle - follow_arm.robot.hand.angle)

165.77000045776367


In [46]:
teach_arm.robot.hand.set_current_limit(-0.1)

In [25]:
teach_arm.robot.hand.set_mode(0)
teach_arm.robot.hand.set_torque(0)

In [43]:
teach_arm.robot.hand.get_current_limit()

5.0

In [23]:
follow_arm.robot.hand.get_current_limit()

5.0

In [ ]:
teach_arm.robot.hand.set_mode(2)
teach_arm.robot.set_enable(True)
teach_arm.robot.hand.set_angle(27)

error in usbbulk_transport.py, process_packet
Traceback (most recent call last):
  File "/Users/jack/Desktop/dummy_ctrl/fibre/usbbulk_transport.py", line 127, in get_packet
    ret = self.epr.read(bufferLen, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 431, in read
    return self.device.read(self, size_or_buffer, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 1043, in read
    ret = fn(
          ^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 850, in bulk_read
    return self.__read(self.lib.libusb_bulk_transfer,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 958, in __read
    _check(retval)
  File "/Use

In [50]:
teach_arm.robot.hand.angle

12.579999923706055

In [4]:
follow_arm = fibre.find_any(serial_number="396636713233", logger=logger)

ConfigurationValue 1
	InterfaceNumber 0,0
		EndpointAddress 130
	InterfaceNumber 1,0
		EndpointAddress 1
		EndpointAddress 129
	InterfaceNumber 2,0
		EndpointAddress 3
		EndpointAddress 131

error in usbbulk_transport.py, process_packet
Traceback (most recent call last):
  File "/Users/jack/Desktop/dummy_ctrl/fibre/usbbulk_transport.py", line 127, in get_packet
    ret = self.epr.read(bufferLen, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 431, in read
    return self.device.read(self, size_or_buffer, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 1043, in read
    ret = fn(
          ^^^
  File "/Users/jack/Desktop/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 850, in bulk_read
    return self.__read(self.lib.libusb_bulk_transfer,
           ^^

In [6]:
print(follow_arm.robot)

calibrate_home_offset()
homing()
resting()
joint_1:
  angle = 0.36375001072883606 (float)
  velocity = 0.0 (float)
  reboot()
  get_temperature()
  set_enable_temperature(enable: bool)
  erase_configs()
  set_enable(enable: bool)
  set_position_with_time(pos: float, time: float)
  set_position(pos: float)
  set_velocity(vel: float)
  set_velocity_limit(vel: float)
  set_current(current: float)
  set_current_limit(current: float)
  set_node_id(id: int)
  set_acceleration(acc: float)
  apply_home_offset()
  do_calibration()
  set_enable_on_boot(enable: bool)
  set_dce_kp(vel: int)
  set_dce_kv(vel: int)
  set_dce_ki(vel: int)
  set_dce_kd(vel: int)
  set_enable_stall_protect(enable: bool)
  update_angle()
joint_2:
  angle = 0.054375000298023224 (float)
  velocity = 0.0 (float)
  reboot()
  get_temperature()
  set_enable_temperature(enable: bool)
  erase_configs()
  set_enable(enable: bool)
  set_position_with_time(pos: float, time: float)
  set_position(pos: float)
  set_velocity(vel: fl

In [7]:
follow_arm.robot.homing()

In [16]:
follow_arm.robot.move_j(10,10,92,5,10,10)

True

In [11]:
follow_arm.robot.set_enable(True)